# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [3]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [5]:
# Constants

#OLLAMA_API = "http://localhost:11434/api/chat"
OLLAMA_API = "http://192.168.0.207:11434/v1"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [6]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [7]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [ ]:
response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [10]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

Generative AI has numerous business applications across various industries, including:

1. **Content Creation**: Generative AI can be used to create personalized content such as product descriptions, social media posts, and even entire articles.
2. **Marketing Automation**: Generative AI can automate tasks like lead generation, email marketing campaigns, and ad creative development.
3. **Customer Service Chatbots**: Generative AI-powered chatbots can provide 24/7 customer support, helping businesses respond to inquiries and resolve issues more efficiently.
4. **Product Design**: Generative AI can be used to design new products, such as product mockups, prototypes, and even entire product lines.
5. **Data Analysis**: Generative AI can help analyze large datasets, identify patterns, and provide insights that can inform business decisions.
6. **Predictive Maintenance**: Generative AI can be used to predict equipment failures, reducing downtime and increasing overall efficiency.
7. **Finan

## Alternative approach - using OpenAI python library to connect to Ollama

In [ ]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI; use either of the above approaches.

In [1]:
# imports
import os
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
from requests.exceptions import RequestException
# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [3]:
# Constants

OLLAMA_API = "http://192.168.0.207:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"
ollama_via_openai = OpenAI(base_url='http://192.168.0.207:11434/v1', api_key='ollama')

In [26]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:

class Website:
    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        self.title = "No title found"
        self.text = "No content found"
        
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        
        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()  # Raises an HTTPError for bad responses
            
            soup = BeautifulSoup(response.content, 'html.parser')
            self.title = soup.title.string if soup.title else "No title found"
            
            # Get text content
            self.text = ' '.join([p.get_text() for p in soup.find_all('p')])
            if not self.text:
                self.text = "No content found"

            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
                
        except requests.exceptions.SSLError:
            raise ConnectionError(f"SSL Certificate verification failed for {url}")
        except requests.exceptions.ConnectionError:
            raise ConnectionError(f"Failed to connect to {url}. Please check if the URL is correct and accessible.")
        except requests.exceptions.Timeout:
            raise ConnectionError(f"Connection timed out while trying to access {url}")
        except requests.exceptions.RequestException as e:
            raise ConnectionError(f"An error occurred while fetching the website: {str(e)}")


In [4]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [5]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [6]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarise these too. Ignoring text that might be navigation related. \n --- \n"
    user_prompt += website.text
    return user_prompt

In [7]:
ms=Website("https://technicallysimple.me")
print(user_prompt_for(ms))

You are looking at a website titled TechnicallySimple.Me – confessions of a techie!
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarise these too. Ignoring text that might be navigation related. 
 --- 
Skip to Content
Looking to 𝗚𝗶𝘃𝗲𝗕𝗮𝗰𝗸 by volunteering my time and skills to help individuals or businesses who can benefit from my life experiences:
𝗟𝗲𝘁'𝘀 𝗰𝗼𝗻𝗻𝗲𝗰𝘁
Home
subscribe
Give Back
About
Contact
TechnicallySimple.Me
confessions of a techie!
TechnicallySimple.Me
confessions of a techie!
Home
subscribe
Give Back
About
Contact
life's hard, tech should be Simple.
Impacting lives through personal experiences & lessons learnt.
Subscribe
How to fix software supply chain integrity with CIS Benchmarks and Aqua Security
Software (application) development lifecycle (SDLC) processes have followed the wild-west approach  where security has been an afterthought to speedy delivery and race to m

In [10]:
# Create a messages list using the same format that we used for OpenAI
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [11]:
me=Website("https://technicallysimple.me")
print(messages_for(me))

NameError: name 'Website' is not defined

In [9]:
def summarise(url):
    try:
        aWebsite = Website(url)
        response = ollama_via_openai.chat.completions.create(
            model=MODEL,
            messages=messages_for(aWebsite)
        )
        return response.choices[0].message.content
    except ConnectionError as e:
        return f"Error: {str(e)}"
    except Exception as e:
        return f"An unexpected error occurred: {str(e)}"

In [31]:
import ollama

def summarise(url):
    try:
        aWebsite = Website(url)
        response = ollama.chat(model=MODEL, messages=messages_for(aWebsite))
        return response['message']['content']
    except ConnectionError as e:
        return f"Error: {str(e)}"
    except Exception as e:
        return f"An unexpected error occurred: {str(e)}"

#print(response['message']['content'])

In [10]:
def display_summary(url):
    try:
        summary = summarise(url)
        display(Markdown(summary))
    except Exception as e:
        print(f"Failed to display summary: {str(e)}")

In [ ]:
address=input("Enter URL: ")
summarise(address)

In [11]:
address=input("Enter URL: ")
display_summary(address)

Wiz appears to be a cloud security platform that offers a range of features and integrations to help organizations protect their cloud workloads. Here are some key points about Wiz:

**Key Features:**

1. **Cloud Security Posture Management (CSPM):** Wiz provides a comprehensive CSPM solution that helps organizations identify and remediate security vulnerabilities in their cloud infrastructure.
2. **Integrations:** Wiz integrates with over 100 cloud providers, including AWS, Azure, Google Cloud Platform, and more.
3. **Security Graphs:** Wiz offers Security Graphs, an innovative feature that enables users to visualize and manage cloud security in a intuitive way.
4. **Bi-directional Sharing:** Wiz's Integration (WIN) platform allows for bi-directional sharing of security findings across the cloud security ecosystem.

**Awards and Recognition:**

1. **Forrester Wave:** Wiz was recognized as a top scorer in The Forrester Wave: Cloud Workload Security, Q1 2024.
2. **Frost & Sullivan:** Wiz was recognized as a leader in Cloud Security Posture Management, 2024.
3. **G2 Grid:** Wiz is a market leader in product, with a 4.7-star rating on G2 Grid for Cloud-Native Application Protection Platforms (CNAPP).

**Customer Testimonials:**

1. **Igor Tsyganskiy:** Global CISO at Bridgewater Associates praised Wiz as "something very right".
2. **Illyan Tytel:** Wiz simplified Azure security, allowing teams to communicate with other teams.

**Resources:**

1. **Blog:** The Wiz blog offers tutorials, research, and insights on cloud security.
2. **CloudSec Academy:** Wiz offers a Cloud Security Awareness platform for training and education.
3. **Customer Stories:** Wiz shares stories of its customers' success with the platform.

Overall, Wiz appears to be a comprehensive cloud security platform that offers innovative features, integrations, and recognition from leading research firms.